# Density-based clustering

Notebook for the density-based clustering

## Autoreload

Autoreload allows the notebook to dynamically load code: if we update some helper functions *outside* of the notebook, we do not need to reload the notebook.

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import procyclingstats as pcs
import umap

import scipy.stats as stats
from scipy.spatial.distance import pdist


## Otherwise nothing will be found
#sys.path.append(os.path.abspath(os.path.join('..')))

#from utility.cluster_utility import scale_data, hier_search, get_average_cyclist_per_cluster